In [28]:
import os
import cv2
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop


In [29]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory(r"C:\Users\HP VICTUS\Desktop\hackathon\TaskA.image\train",
                                          target_size=(200,200),
                                          batch_size = 32,
                                          class_mode = 'binary',
                                          shuffle = True)
                                         
test_dataset = test.flow_from_directory(r"C:\Users\HP VICTUS\Desktop\hackathon\TaskA.image\val",
                                          target_size=(200,200),
                                          batch_size =32,
                                          class_mode = 'binary',
                                          shuffle = False)

Found 2337 images belonging to 2 classes.
Found 406 images belonging to 2 classes.


In [30]:
test_dataset.class_indices

{'female': 0, 'male': 1}

In [ ]:
model = keras.Sequential([
    # Block 1
    keras.layers.ZeroPadding2D(padding=(3, 3), input_shape=(200, 200, 3)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),

    # Block 2
    keras.layers.ZeroPadding2D(padding=(3, 3)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),

    # Block 3
    keras.layers.ZeroPadding2D(padding=(3, 3)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),

    # Block 4
    keras.layers.ZeroPadding2D(padding=(3, 3)),
    keras.layers.Conv2D(512, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),

    # Classification Head
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [32]:
model.compile(
    optimizer=RMSprop(learning_rate=0.001, rho=0.9),
    loss='binary_crossentropy',
    metrics=['accuracy']
)



In [34]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint("model.h5", save_best_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)


In [36]:
steps_per_epoch = train_dataset.samples // train_dataset.batch_size
validation_steps = test_dataset.samples // test_dataset.batch_size

history = model.fit(
    train_dataset,
    epochs=14,  # Increase for real training
    validation_data=test_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stop, checkpoint, lr_reduce]
)

model.load_weights("model.h5")
# 4. Evaluate the Model
test_dataset.reset()  # Important for correct predictions
pred_probs = model.predict(test_dataset)
preds = (pred_probs > 0.5).astype(int).flatten()
true_labels = test_dataset.classes

print("\n--- Evaluation Metrics ---")
print("Classification Report:\n", classification_report(true_labels, preds))
print("Accuracy:", accuracy_score(true_labels, preds))
print("Precision:", precision_score(true_labels, preds))
print("Recall:", recall_score(true_labels, preds))
print("F1 Score:", f1_score(true_labels, preds))

# 5. Save the Model
model.save("model.h5")


Epoch 1/14
73/73 [==============================] - ETA: 0s - loss: 0.5208 - accuracy: 0.8043

c:\Users\HP VICTUS\anaconda3\envs\h\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


73/73 [==============================] - 384s 5s/step - loss: 0.5208 - accuracy: 0.8043 - val_loss: 0.4613 - val_accuracy: 0.7995 - lr: 0.0010
Epoch 2/14
73/73 [==============================] - 1145s 16s/step - loss: 0.4127 - accuracy: 0.8043 - val_loss: 0.4659 - val_accuracy: 0.8021 - lr: 0.0010
Epoch 3/14
73/73 [==============================] - 1244s 17s/step - loss: 0.3745 - accuracy: 0.8169 - val_loss: 0.3774 - val_accuracy: 0.8203 - lr: 0.0010
Epoch 4/14
73/73 [==============================] - 375s 5s/step - loss: 0.3523 - accuracy: 0.8416 - val_loss: 0.3421 - val_accuracy: 0.8568 - lr: 0.0010
Epoch 5/14
73/73 [==============================] - 1487s 21s/step - loss: 0.3273 - accuracy: 0.8586 - val_loss: 0.4551 - val_accuracy: 0.7656 - lr: 0.0010
Epoch 6/14
73/73 [==============================] - 320s 4s/step - loss: 0.2849 - accuracy: 0.8797 - val_loss: 0.2920 - val_accuracy: 0.8698 - lr: 0.0010
Epoch 7/14
73/73 [==============================] - 243s 3s/step - loss: 0.2655 -

c:\Users\HP VICTUS\anaconda3\envs\h\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


✅ Model saved as 'gender_model.h5'
